In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize 

In [3]:
df = pd.read_csv("movie.csv")
df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [13]:
df2 = df[df["release_year"] >=2020]
df2.shape

(1972, 15)

In [15]:
df2 = df2[df["type"] == "MOVIE"]
df2.shape

C:\Users\mac\AppData\Local\Temp\ipykernel_20444\1795181273.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = df2[df["type"] == "MOVIE"]


(1173, 15)

In [18]:
columns = ["seasons", "imdb_id", "age_certification", "id", "tmdb_score", "tmdb_popularity", "imdb_votes"]
df2.drop(columns, axis = 1, inplace=True)
df2.head(20)

C:\Users\mac\AppData\Local\Temp\ipykernel_20444\2019155080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(columns, axis = 1, inplace=True)


,title,type,description,release_year,runtime,genres,production_countries,imdb_score
3072,The Rental,MOVIE,Two couples rent a vacation home for what shou...,2020,88,"['drama', 'horror', 'thriller']",['US'],5.7
3075,I'm Thinking of Ending Things,MOVIE,Nothing is as it seems when a woman experienci...,2020,134,"['drama', 'thriller', 'fantasy']",['US'],6.6
3082,Extraction,MOVIE,"Tyler Rake, a fearless black market mercenary,...",2020,116,"['crime', 'drama', 'action', 'thriller']",['US'],6.7
3084,My Octopus Teacher,MOVIE,After years of swimming every day in the freez...,2020,84,['documentation'],"['ZA', 'GB']",8.1
3087,"Yes, God, Yes",MOVIE,"Alice, a young innocent Catholic girl, is temp...",2020,78,"['comedy', 'drama']","['GB', 'US']",6.1
3090,Enola Holmes,MOVIE,"While searching for her missing mother, intrep...",2020,123,"['crime', 'drama', 'action']",['US'],6.6
3092,The Old Guard,MOVIE,Four undying warriors who've secretly protecte...,2020,125,"['fantasy', 'action', 'thriller']",['US'],6.6
3094,His House,MOVIE,After making a harrowing escape from war-torn ...,2020,93,"['thriller', 'drama', 'horror']",['GB'],6.5
3098,The Devil All the Time,MOVIE,"In Knockemstiff, Ohio and its neighboring back...",2020,138,"['crime', 'drama', 'thriller', 'horror']",['US'],7.1
3111,Summerland,MOVIE,A woman during the Second World War opens her ...,2020,102,"['romance', 'war', 'drama']",['GB'],7.0


In [19]:
df2.to_csv("processed_movie.csv")

In [20]:
df2["description"].iloc[0]

'Two couples rent a vacation home for what should be a celebratory weekend get-away.'

In [21]:
#process description column, create a new column that joins the description with genre column. Use that column to apply tfidfvectorizer.
import nltk
import re
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
df2["description"] = df2["description"].apply(clean)
df2["description"].iloc[0]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\mac\AppData\Local\Temp\ipykernel_20444\2900106977.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["description"] = df2["description"].apply(clean)


'two coupl rent vacat home celebratori weekend getaway'

In [22]:
df2 = df2.drop('type', axis = 1)
df2.head()

,title,description,release_year,runtime,genres,production_countries,imdb_score
3072,The Rental,two coupl rent vacat home celebratori weekend ...,2020,88,"['drama', 'horror', 'thriller']",['US'],5.7
3075,I'm Thinking of Ending Things,noth seem woman experienc misgiv new boyfriend...,2020,134,"['drama', 'thriller', 'fantasy']",['US'],6.6
3082,Extraction,tyler rake fearless black market mercenari emb...,2020,116,"['crime', 'drama', 'action', 'thriller']",['US'],6.7
3084,My Octopus Teacher,year swim everi day freez ocean tip africa cra...,2020,84,['documentation'],"['ZA', 'GB']",8.1
3087,"Yes, God, Yes",alic young innoc cathol girl tempt masturb aol...,2020,78,"['comedy', 'drama']","['GB', 'US']",6.1


In [23]:
df2.to_csv("processed_movie.csv")

In [26]:
df2 = pd.read_csv("processed_movie.csv")
df2.head()

,Unnamed: 0,title,description,release_year,runtime,genres,production_countries,imdb_score
0,3072,The Rental,two coupl rent vacat home celebratori weekend ...,2020,88,"['drama', 'horror', 'thriller']",['US'],5.7
1,3075,I'm Thinking of Ending Things,noth seem woman experienc misgiv new boyfriend...,2020,134,"['drama', 'thriller', 'fantasy']",['US'],6.6
2,3082,Extraction,tyler rake fearless black market mercenari emb...,2020,116,"['crime', 'drama', 'action', 'thriller']",['US'],6.7
3,3084,My Octopus Teacher,year swim everi day freez ocean tip africa cra...,2020,84,['documentation'],"['ZA', 'GB']",8.1
4,3087,"Yes, God, Yes",alic young innoc cathol girl tempt masturb aol...,2020,78,"['comedy', 'drama']","['GB', 'US']",6.1


In [27]:
df2.drop("Unnamed: 0", axis=1, inplace=True)
df2.head()

,title,description,release_year,runtime,genres,production_countries,imdb_score
0,The Rental,two coupl rent vacat home celebratori weekend ...,2020,88,"['drama', 'horror', 'thriller']",['US'],5.7
1,I'm Thinking of Ending Things,noth seem woman experienc misgiv new boyfriend...,2020,134,"['drama', 'thriller', 'fantasy']",['US'],6.6
2,Extraction,tyler rake fearless black market mercenari emb...,2020,116,"['crime', 'drama', 'action', 'thriller']",['US'],6.7
3,My Octopus Teacher,year swim everi day freez ocean tip africa cra...,2020,84,['documentation'],"['ZA', 'GB']",8.1
4,"Yes, God, Yes",alic young innoc cathol girl tempt masturb aol...,2020,78,"['comedy', 'drama']","['GB', 'US']",6.1


In [28]:
df2.to_csv("processed_movie.csv")

In [34]:
df2['genre2'] = df2['genres'].apply(lambda x: x[1:-1])
df2.head()

,title,description,release_year,runtime,genres,production_countries,imdb_score,genre2
0,The Rental,two coupl rent vacat home celebratori weekend ...,2020,88,"['drama', 'horror', 'thriller']",['US'],5.7,"'drama', 'horror', 'thriller'"
1,I'm Thinking of Ending Things,noth seem woman experienc misgiv new boyfriend...,2020,134,"['drama', 'thriller', 'fantasy']",['US'],6.6,"'drama', 'thriller', 'fantasy'"
2,Extraction,tyler rake fearless black market mercenari emb...,2020,116,"['crime', 'drama', 'action', 'thriller']",['US'],6.7,"'crime', 'drama', 'action', 'thriller'"
3,My Octopus Teacher,year swim everi day freez ocean tip africa cra...,2020,84,['documentation'],"['ZA', 'GB']",8.1,'documentation'
4,"Yes, God, Yes",alic young innoc cathol girl tempt masturb aol...,2020,78,"['comedy', 'drama']","['GB', 'US']",6.1,"'comedy', 'drama'"


In [41]:
def process(text):
    text = [word[1:-1] for word in text.split(', ')]
    text=" ".join(text)
    return text
df2['genre2'] = df2['genres'].apply(process)
df2.head()

,title,description,release_year,runtime,genres,production_countries,imdb_score,genre2
0,The Rental,two coupl rent vacat home celebratori weekend ...,2020,88,"['drama', 'horror', 'thriller']",['US'],5.7,'drama horror thriller'
1,I'm Thinking of Ending Things,noth seem woman experienc misgiv new boyfriend...,2020,134,"['drama', 'thriller', 'fantasy']",['US'],6.6,'drama thriller fantasy'
2,Extraction,tyler rake fearless black market mercenari emb...,2020,116,"['crime', 'drama', 'action', 'thriller']",['US'],6.7,'crime drama action thriller'
3,My Octopus Teacher,year swim everi day freez ocean tip africa cra...,2020,84,['documentation'],"['ZA', 'GB']",8.1,'documentation'
4,"Yes, God, Yes",alic young innoc cathol girl tempt masturb aol...,2020,78,"['comedy', 'drama']","['GB', 'US']",6.1,'comedy drama'


In [43]:
df2['genre2'] = df2['genre2'].str[1: -1]
df2.head()

,title,description,release_year,runtime,genres,production_countries,imdb_score,genre2
0,The Rental,two coupl rent vacat home celebratori weekend ...,2020,88,"['drama', 'horror', 'thriller']",['US'],5.7,drama horror thriller
1,I'm Thinking of Ending Things,noth seem woman experienc misgiv new boyfriend...,2020,134,"['drama', 'thriller', 'fantasy']",['US'],6.6,drama thriller fantasy
2,Extraction,tyler rake fearless black market mercenari emb...,2020,116,"['crime', 'drama', 'action', 'thriller']",['US'],6.7,crime drama action thriller
3,My Octopus Teacher,year swim everi day freez ocean tip africa cra...,2020,84,['documentation'],"['ZA', 'GB']",8.1,documentation
4,"Yes, God, Yes",alic young innoc cathol girl tempt masturb aol...,2020,78,"['comedy', 'drama']","['GB', 'US']",6.1,comedy drama


In [47]:
df2['overall'] = df2['description'] +" " + df2['genre2']
df2["overall"].iloc[0]

'two coupl rent vacat home celebratori weekend getaway drama horror thriller'

In [49]:
columns = ["title", "description", "genre2", "overall"]
df3 = df2[columns]
df3.head()

,title,description,genre2,overall
0,The Rental,two coupl rent vacat home celebratori weekend ...,drama horror thriller,two coupl rent vacat home celebratori weekend ...
1,I'm Thinking of Ending Things,noth seem woman experienc misgiv new boyfriend...,drama thriller fantasy,noth seem woman experienc misgiv new boyfriend...
2,Extraction,tyler rake fearless black market mercenari emb...,crime drama action thriller,tyler rake fearless black market mercenari emb...
3,My Octopus Teacher,year swim everi day freez ocean tip africa cra...,documentation,year swim everi day freez ocean tip africa cra...
4,"Yes, God, Yes",alic young innoc cathol girl tempt masturb aol...,comedy drama,alic young innoc cathol girl tempt masturb aol...


In [58]:
df3.to_csv("recommend_data.csv", index=False)

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
df3 = df3[~df3["overall"].isnull()]
df3.shape

(1169, 4)

In [59]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df3["overall"])
tfidf_matrix

<1169x5955 sparse matrix of type '<class 'numpy.float64'>'
	with 23057 stored elements in Compressed Sparse Row format>

In [60]:
similarity = cosine_similarity(tfidf_matrix)
similarity

array([[1.        , 0.02853128, 0.0212593 , ..., 0.00951563, 0.        ,
        0.        ],
       [0.02853128, 1.        , 0.01869245, ..., 0.00836671, 0.02940897,
        0.        ],
       [0.0212593 , 0.01869245, 1.        , ..., 0.00623422, 0.0392583 ,
        0.        ],
       ...,
       [0.00951563, 0.00836671, 0.00623422, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.02940897, 0.0392583 , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [61]:
df3.head()

,title,description,genre2,overall
0,The Rental,two coupl rent vacat home celebratori weekend ...,drama horror thriller,two coupl rent vacat home celebratori weekend ...
1,I'm Thinking of Ending Things,noth seem woman experienc misgiv new boyfriend...,drama thriller fantasy,noth seem woman experienc misgiv new boyfriend...
2,Extraction,tyler rake fearless black market mercenari emb...,crime drama action thriller,tyler rake fearless black market mercenari emb...
3,My Octopus Teacher,year swim everi day freez ocean tip africa cra...,documentation,year swim everi day freez ocean tip africa cra...
4,"Yes, God, Yes",alic young innoc cathol girl tempt masturb aol...,comedy drama,alic young innoc cathol girl tempt masturb aol...


In [71]:
df3 = df3.reset_index( drop=True)

In [73]:
indices = pd.Series(df3.index, 
                    index=df3['title'])
indices

title
The Rental                                      0
I'm Thinking of Ending Things                   1
Extraction                                      2
My Octopus Teacher                              3
Yes, God, Yes                                   4
                                             ... 
Bling Empire - The Afterparty                1164
Fine Wine                                    1165
C/O Kaadhal                                  1166
Lokillo                                      1167
Dad Stop Embarrassing Me - The Afterparty    1168
Length: 1169, dtype: int64

In [75]:
def netFlix_recommendation(title, similarity = similarity):
    index = indices[title]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:11]
    movieindices = [i[0] for i in similarity_scores]
    return df3['title'].iloc[movieindices]

print(netFlix_recommendation("Lokillo"))

700       Shiny_Flakes: The Teenage Drug Lord
1067    The Misadventures of Hedi and Cokeman
1107                  Loyiso Gola: Unlearning
804                                     Ferry
345                           Earth and Blood
607                                  Centauro
677             Jimmy Carr: His Dark Material
205                             Bulletproof 2
103                              The Stand In
425                                   Santana
Name: title, dtype: object
